# Running lots of BLASTs
Run BLASTs on 8 ENTREZ IDs and store in hits data frame

In [63]:
import math
import os
import requests

from Bio.Blast import NCBIWWW
from Bio.Blast import NCBIXML
import numpy as np
import pandas as pd

### Create pandas data frame for results, manually add ENTREZID's

In [57]:
data1 = pd.read_csv('data/001.csv')
#Make sure ENTREZID column is integers not floats, or else NCBI cannot read it
eids001 = [939978, 939970, 939444, 939454, 939466, 939456, 940002, 939969, 938032, 940000, 938032]

selected_columns = data1[['GENENAME', 'ENTREZID', 'Log2fc_(GC)v(FLT)', 'P.value_(GC)v(FLT)', 'Adj.p.value_(GC)v(FLT)']]
bioproject_df = pd.DataFrame(selected_columns)[0:10]
protein_df = pd.DataFrame(selected_columns)[0:10]

In [36]:
bioproject_df

,GENENAME,ENTREZID,Log2fc_(GC)v(FLT),P.value_(GC)v(FLT),Adj.p.value_(GC)v(FLT)
0,chromosomal replication initiator informationa...,939978.0,0.320344,9.332727e-05,4.157477e-04
1,DNA polymerase III (beta subunit),939970.0,0.578609,5.564729e-10,7.000177e-09
2,RNA binding protein involved in ribosome matur...,939444.0,0.295755,1.011347e-04,4.478243e-04
3,"RecA filament-DNA complex stabilisation, ssDNA...",939454.0,-0.047216,4.950332e-01,6.112680e-01
4,regulator of extracellular matrix formation,939466.0,0.043596,5.978884e-01,6.994584e-01
5,DNA gyrase (subunit B),939456.0,0.079820,2.375809e-01,3.488090e-01
6,DNA gyrase (subunit A),940002.0,0.042476,6.033868e-01,7.036599e-01
7,hypothetical protein,939969.0,0.572560,3.194062e-03,9.416315e-03
8,inosine-monophosphate dehydrogenase,938032.0,0.092622,5.086308e-01,6.225034e-01
9,D-alanyl-D-alanine carboxypeptidase (penicilli...,940000.0,0.398396,3.413016e-04,1.303955e-03


In [52]:
# --Doesn't work
def create_hits_df(db_hits_eidindex):
    """
    Creates a dataframe for the NCBI BLAST hits results
    """
    db_hits_eidindex = pd.DataFrame()
    db_hits_eidindex['hits_EID'] = ''
    db_hits_eidindex['hits_e_value'] = ''
    db_hits_eidindex['hits_score'] = ''
    db_hits_eidindex['normalized_score_%'] = ''
    return db_hits_eidindex

In [53]:
# Initiate results dataframes -- Doesn't work

create_hits_df('protein_hits_0')
create_hits_df('protein_hits_1')
create_hits_df('protein_hits_2')
create_hits_df('protein_hits_3')
create_hits_df('protein_hits_4')
create_hits_df('protein_hits_5')
create_hits_df('protein_hits_6')
create_hits_df('protein_hits_7')

create_hits_df('pdb_hits_0')
create_hits_df('pdb_hits_1')
create_hits_df('pdb_hits_2')
create_hits_df('pdb_hits_3')
create_hits_df('pdb_hits_4')
create_hits_df('pdb_hits_5')
create_hits_df('pdb_hits_6')
create_hits_df('pdb_hits_7')

create_hits_df('bioproject_hits_0')
create_hits_df('bioproject_hits_1')
create_hits_df('bioproject_hits_2')
create_hits_df('bioproject_hits_3')
create_hits_df('bioproject_hits_4')
create_hits_df('bioproject_hits_5')
create_hits_df('bioproject_hits_6')
create_hits_df('bioproject_hits_7')

,hits_EID,hits_e_value,hits_score,normalized_score_%


In [66]:
# Create hits databases manually filled with zeros

column_names = ['hits_EID', 'hits_e_value', 'hits_score', 'normalized_score_%']

protein_hits_0 = pd.DataFrame(np.zeros((50, 4)), columns=column_names)
protein_hits_1 = pd.DataFrame(np.zeros((50, 4)), columns=column_names)
protein_hits_2 = pd.DataFrame(np.zeros((50, 4)), columns=column_names)
protein_hits_3 = pd.DataFrame(np.zeros((50, 4)), columns=column_names)
protein_hits_4 = pd.DataFrame(np.zeros((50, 4)), columns=column_names)
protein_hits_5 = pd.DataFrame(np.zeros((50, 4)), columns=column_names)
protein_hits_6 = pd.DataFrame(np.zeros((50, 4)), columns=column_names)
protein_hits_7 = pd.DataFrame(np.zeros((50, 4)), columns=column_names)

pdb_hits_0 = pd.DataFrame(np.zeros((50, 4)), columns=column_names)
pdb_hits_1 = pd.DataFrame(columns=column_names)
pdb_hits_2 = pd.DataFrame(columns=column_names)
pdb_hits_3 = pd.DataFrame(columns=column_names)
pdb_hits_4 = pd.DataFrame(columns=column_names)
pdb_hits_5 = pd.DataFrame(columns=column_names)
pdb_hits_6 = pd.DataFrame(columns=column_names)
pdb_hits_7 = pd.DataFrame(columns=column_names)

bioproject_hits_0 = pd.DataFrame(columns=column_names)
bioproject_hits_1 = pd.DataFrame(columns=column_names)
bioproject_hits_2 = pd.DataFrame(columns=column_names)
bioproject_hits_3 = pd.DataFrame(columns=column_names)
bioproject_hits_4 = pd.DataFrame(columns=column_names)
bioproject_hits_5 = pd.DataFrame(columns=column_names)
bioproject_hits_6 = pd.DataFrame(columns=column_names)
bioproject_hits_7 = pd.DataFrame(columns=column_names)

### Function to BLAST for EID's and save in data frame per NCBI DB as .XML and convert pd.DF to .csv

In [39]:
def blast_eid_for_dataframe(ENTREZID, save_file_xml, eid_index, ncbi_database, results_dataframe, csv):
    """
    Function: Take EID, BLAST in a given NCBI database
    ENTREZID: ENTREZID from 001.csv as added to bioproject_df, float
    save_file_xml: file to save to as ncbidb_index.xml, string
    eid_index: index of the ENTREZID, float
    ncbi_database: the NCBI database to be used for BLASTing, string
    results_dataframe: ncbi_hits, as created previously, string
    csv: filename as ncbidb_index.csv to store hits into, string
    """
    eid = ENTREZID
    result_handle = NCBIWWW.qblast('blastp', ncbi_database, eid)
    with open(save_file_xml, 'w+') as save_to:
        save_to.write(result_handle.read())
        result_handle.close()    
    result_handle = open(save_file_xml)
    blast_record = NCBIXML.read(result_handle)

    E_VALUE_THRESH = 0.0000000000000000000000000000000000000000000000004
    for alignment in blast_record.alignments:
        for hsp in alignment.hsps:
            if hsp.expect < E_VALUE_THRESH:
                results_dataframe['hits_EID'][eid_index] = alignment.title
                results_dataframe['hits_e_value'][eid_index] = hsp.expect
                results_dataframe['hits_score'][eid_index] = hsp.score
                results_dataframe['normalized_score_%'][eid_index] = hsp.score / results_dataframe['normalized_score_%'][1] * 100
    
    results_dataframe.to_csv(csv)
    return 

Test this function  
Results: it seems to have been running

In [75]:
result_handle = open('pdb_0.xml')
blast_record = NCBIXML.read(result_handle)

E_VALUE_THRESH = 0.0000000000000000000000000000000000000000000000004
for alignment in blast_record.alignments:
    for hsp in alignment.hsps:
        if hsp.expect < E_VALUE_THRESH:
            print(alignment.title)
            print(hsp.expect)

pdb|1HDG|O The Crystal Structure Of Holo-glyceraldehyde-3-phosphate Dehydrogenase From The Hyperthermophilic Bacterium Thermotoga Maritima At 2.5 Angstroms Resolution [Thermotoga maritima] >pdb|1HDG|Q The Crystal Structure Of Holo-glyceraldehyde-3-phosphate Dehydrogenase From The Hyperthermophilic Bacterium Thermotoga Maritima At 2.5 Angstroms Resolution [Thermotoga maritima]
0.0
pdb|1GD1|O Structure Of Holo-Glyceraldehyde-3-Phosphate Dehydrogenase From Bacillus Stearothermophilus At 1.8 Angstroms Resolution [Geobacillus stearothermophilus] >pdb|1GD1|P Structure Of Holo-Glyceraldehyde-3-Phosphate Dehydrogenase From Bacillus Stearothermophilus At 1.8 Angstroms Resolution [Geobacillus stearothermophilus] >pdb|1GD1|Q Structure Of Holo-Glyceraldehyde-3-Phosphate Dehydrogenase From Bacillus Stearothermophilus At 1.8 Angstroms Resolution [Geobacillus stearothermophilus] >pdb|1GD1|R Structure Of Holo-Glyceraldehyde-3-Phosphate Dehydrogenase From Bacillus Stearothermophilus At 1.8 Angstroms Re

In [74]:
result_handle = open('pdb_0.xml')
blast_record = NCBIXML.read(result_handle)

E_VALUE_THRESH = 0.0000000000000000000000000000000000000000000000004
for alignment in blast_record.alignments:
    for hsp in alignment.hsps:
        if hsp.expect < E_VALUE_THRESH:
            pdb_hits_0.loc[alignment,('hits_EID')] = alignment.title
            pdb_hits_0.loc[alignment,('hits_EID')] = hsp.expect
            pdb_hits_0.loc[alignment,('hits_EID')] = hsp.score
            pdb_hits_0.loc[alignment,('hits_EID')] = hsp.score / pdb_hits_0['normalized_score_%'][0] * 100
    
pdb_hits_0.to_csv('pdb_1.csv')

TypeError: int() argument must be a string, a bytes-like object or a number, not 'Alignment'

In [71]:
result_handle = open('pdb_0.xml')
blast_record = NCBIXML.read(result_handle)

E_VALUE_THRESH = 0.0000000000000000000000000000000000000000000000004
for alignment in blast_record.alignments:
    for hsp in alignment.hsps:
        if hsp.expect < E_VALUE_THRESH:
            pdb_hits_0['hits_EID'][alignment] = alignment.title
            pdb_hits_0['hits_e_value'][alignment] = hsp.expect
            pdb_hits_0['hits_score'][alignment] = hsp.score
            pdb_hits_0['normalized_score_%'][alignment] = hsp.score / pdb_hits_0['normalized_score_%'][0] * 100
    
pdb_hits_0.to_csv('pdb_1.csv')

<ipython-input-71-d379b6bdde6b>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pdb_hits_0['hits_EID'][alignment] = alignment.title


TypeError: int() argument must be a string, a bytes-like object or a number, not 'Alignment'

In [60]:
eids001[1]

939970

In [59]:
blast_eid_for_dataframe(eids001[1], 'pdb_1.xml', 1, 'pdb', pdb_hits_1, 'pdb_1.csv')

IndexError: index 1 is out of bounds for axis 0 with size 0

### BLAST the first 8 ENTREZIDs for 3 different databases

In [41]:
blast_eid_for_dataframe(eids001[0], 'protein_0.xml', 0, 'protein', 'protein_hits_0', 'protein_0.csv')
blast_eid_for_dataframe(eids001[1], 'protein_1.xml', 1, 'protein', 'protein_hits_1', 'protein_1.csv')
blast_eid_for_dataframe(eids001[2], 'protein_2.xml', 2, 'protein', 'protein_hits_2', 'protein_2.csv')
blast_eid_for_dataframe(eids001[3], 'protein_3.xml', 3, 'protein', 'protein_hits_3', 'protein_3.csv')
blast_eid_for_dataframe(eids001[4], 'protein_4.xml', 4, 'protein', 'protein_hits_4', 'protein_4.csv')
blast_eid_for_dataframe(eids001[5], 'protein_5.xml', 5, 'protein', 'protein_hits_5', 'protein_5.csv')
blast_eid_for_dataframe(eids001[6], 'protein_6.xml', 6, 'protein', 'protein_hits_6', 'protein_6.csv')
blast_eid_for_dataframe(eids001[7], 'protein_7.xml', 7, 'protein', 'protein_hits_7', 'protein_7.csv')

blast_eid_for_dataframe(eids001[0], 'pdb_0.xml', 0, 'pdb', 'pdb_hits_0', 'pdb_0.csv')
blast_eid_for_dataframe(eids001[1], 'pdb_1.xml', 1, 'pdb', 'pdb_hits_1', 'pdb_1.csv')
blast_eid_for_dataframe(eids001[2], 'pdb_2.xml', 2, 'pdb', 'pdb_hits_2', 'pdb_2.csv')
blast_eid_for_dataframe(eids001[3], 'pdb_3.xml', 3, 'pdb', 'pdb_hits_3', 'pdb_3.csv')
blast_eid_for_dataframe(eids001[4], 'pdb_4.xml', 4, 'pdb', 'pdb_hits_4', 'pdb_4.csv')
blast_eid_for_dataframe(eids001[5], 'pdb_5.xml', 5, 'pdb', 'pdb_hits_5', 'pdb_5.csv')
blast_eid_for_dataframe(eids001[6], 'pdb_6.xml', 6, 'pdb', 'pdb_hits_6', 'pdb_6.csv')
blast_eid_for_dataframe(eids001[7], 'pdb_7.xml', 7, 'pdb', 'pdb_hits_7', 'pdb_7.csv')

blast_eid_for_dataframe(eids001[0], 'bioproject_0.xml', 0, 'bioproject', 'bioproject_hits_0', 'bioproject_0.csv')
blast_eid_for_dataframe(eids001[1], 'bioproject_1.xml', 1, 'bioproject', 'bioproject_hits_1', 'bioproject_1.csv')
blast_eid_for_dataframe(eids001[2], 'bioproject_2.xml', 2, 'bioproject', 'bioproject_hits_2', 'bioproject_2.csv')
blast_eid_for_dataframe(eids001[3], 'bioproject_3.xml', 3, 'bioproject', 'bioproject_hits_3', 'bioproject_3.csv')
blast_eid_for_dataframe(eids001[4], 'bioproject_4.xml', 4, 'bioproject', 'bioproject_hits_4', 'bioproject_4.csv')
blast_eid_for_dataframe(eids001[5], 'bioproject_5.xml', 5, 'bioproject', 'bioproject_hits_5', 'bioproject_5.csv')
blast_eid_for_dataframe(eids001[6], 'bioproject_6.xml', 6, 'bioproject', 'bioproject_hits_6', 'bioproject_6.csv')
blast_eid_for_dataframe(eids001[7], 'bioproject_7.xml', 7, 'bioproject', 'bioproject_hits_7', 'bioproject_7.csv')

AttributeError: 'str' object has no attribute 'to_csv'